<h3>Import necessary libararies

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
import pandas as pd


This code prevents the kernel from stopping when XGBoost is running

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

<h3>Load in DF

In [8]:
df = pd.read_csv('df_english_articles.csv')

FileNotFoundError: [Errno 2] File b'df_english_articles.csv' does not exist: b'df_english_articles.csv'

In [20]:
df.shape

(95790, 13)

In [21]:
df = df.drop(['Unnamed: 0'], axis = 1)

In [22]:
df.head()

,title,content,category,gensim_summary,first_100,sent_tokenized,tokenized_first_100,first_100_no_stops,lemmatize_first_100,language,stemmed,kmeans12
0,Agent Cooper in Twin Peaks is the audience: on...,And never more so than in Showtime’s new...,Longform,"In the second season finale, back in 1991, the...",and never more so than in showtime’s new serie...,[' And never more so than in Showtime’s n...,"['and', 'never', 'more', 'so', 'than', 'in', '...","['never', 'showtime', 'new', 'series', 'reviva...",never showtime new series revival spoiler ahea...,en,never showtim new seri reviv spoiler ahead epi...,3
1,"AI, the humanity!",AlphaGo’s victory isn’t a defeat for hum...,Longform,When speaking to DeepMind and Google developer...,alphago’s victory isn’t a defeat for humans — ...,[' AlphaGo’s victory isn’t a defeat for h...,"['alphago', '’', 's', 'victory', 'isn', '’', '...","['alphago', 'victory', 'defeat', 'humans', 'op...",alphago victory defeat human opportunity loss ...,en,alphago victori defeat human opportun loss hum...,3
2,Massive attack,How a weapon against war became a weapon...,Longform,International visitors for the event are commo...,how a weapon against war became a weapon again...,[' How a weapon against war became a weap...,"['how', 'a', 'weapon', 'against', 'war', 'beca...","['weapon', 'war', 'became', 'weapon', 'web', '...",weapon war became weapon web every year artist...,en,weapon war becam weapon web everi year artist ...,3
3,Brain drain,Genius quietly laid off a bunch of its e...,Longform,"In a post on the Genius blog at the time, co-f...",genius quietly laid off a bunch of its enginee...,[' Genius quietly laid off a bunch of its...,"['genius', 'quietly', 'laid', 'off', 'a', 'bun...","['genius', 'quietly', 'laid', 'bunch', 'engine...",genius quietly laid bunch engineer survive med...,en,geniu quietli laid bunch engin surviv media co...,10
4,Facebook takes flight,Inside the test flight of Facebook’s fir...,Longform,But if your goal is to stay in the air for a l...,inside the test flight of facebook’s first int...,[' Inside the test flight of Facebook’s f...,"['inside', 'the', 'test', 'flight', 'of', 'fac...","['inside', 'test', 'flight', 'facebook', 'firs...",inside test flight facebook first internet dro...,en,insid test flight facebook first internet dron...,3


<h3>Check for null values

In [24]:
df.isnull().sum()

title                      0
content                    0
category               12097
gensim_summary             0
first_100                  0
sent_tokenized             0
tokenized_first_100        0
first_100_no_stops         0
lemmatize_first_100        0
language                   0
stemmed                    0
kmeans12                   0
dtype: int64

In [25]:
#df = df.dropna(subset = ['kmeans12'])

In [26]:
df.shape

(95790, 12)

<h3>Label Encoding

In [28]:
# Create a label encoder object
le = preprocessing.LabelEncoder()

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95790 entries, 0 to 95789
Data columns (total 12 columns):
title                  95790 non-null object
content                95790 non-null object
category               83693 non-null object
gensim_summary         95790 non-null object
first_100              95790 non-null object
sent_tokenized         95790 non-null object
tokenized_first_100    95790 non-null object
first_100_no_stops     95790 non-null object
lemmatize_first_100    95790 non-null object
language               95790 non-null object
stemmed                95790 non-null object
kmeans12               95790 non-null int64
dtypes: int64(1), object(11)
memory usage: 8.8+ MB


In [31]:
# Fit the encoder to the pandas column
le.fit(df.kmeans12)

LabelEncoder()

In [32]:
targets = list(le.classes_)
print(targets)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [33]:
# Apply the fitted encoder to the pandas column
le.transform(df['kmeans12']) 

array([3, 3, 3, ..., 3, 3, 3])

<h3>Train Test Split

In [72]:
X = df.content
y = df.kmeans12

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

<h3>Naive Bayes Classifier

In [35]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [40]:
%%time

y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
#print(classification_report(y_test, y_pred,target_names = targets))

accuracy 0.4917005950516755
CPU times: user 33.8 s, sys: 596 ms, total: 34.4 s
Wall time: 34.7 s


In [67]:
X_test_2 = ["""he train populist itali foment revolut belgium He texa build wall but halt project placehold the real question whether steve bannon will back donald trump side 2020 two-year exil trumpworld that could happen least accord trump tell one thing trump said ask bannon februari dure oval offic interview watch bannon time four five time last six month nobodi say anyth better right bannon know. bannon cours mastermind took falter trump campaign august 2016 guid improb victori He serv trump chief polit strategist white hous forc trump second chief staff john kelli in move plainli exhaust bannon seem almost welcom month battl presid son-in-law jare kushner presid daughter ivanka trump well pretti much everyon els"""]

In [68]:
new_pred = nb.predict(X_test_2)

In [69]:
new_pred

array([1])

<h3>Logistic Regression Classifier

In [37]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)

/Users/austinkrause/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/austinkrause/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False))])

In [39]:
%%time

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
#print(classification_report(y_test, y_pred,target_names=targets))

accuracy 0.740299961721822
CPU times: user 33.3 s, sys: 547 ms, total: 33.9 s
Wall time: 34 s


<h3>Random Forest Classifier --- STILL NEEDS TO BE RUN 

In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
# text_classifier = RandomForestClassifier(n_estimators=200, random_state=0)  
# text_classifier.fit(X_train, y_train)

In [60]:
#set up paramaters dictionary for grid search
param_grid = { 
    'n_estimators': [100, 200,300],
    'max_features': ['auto',0.25, 0.33, 0.5],
    'max_depth' : [None,5,6,7,8],
    'min_samples_leaf': [0.03,0.04,0.05, 1, 2]
    }

In [61]:
rfc = RandomForestClassifier()

In [62]:
#create instance of gridsearchCV
#use params from above
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 3,n_jobs=-1)

In [1]:
#f.fit(X_train, y_train)

In [ ]:
#check best paramaters from the grid search
CV_rfc.best_params_

In [ ]:
#use model to predict on test set
rfc_pred = CV_rfc.best_estimator_.predict(X_test)
#check accuracy on test set
print('Test Accuracy score: ', accuracy_score(y_test, rfc_pred))
#check F1 of test set
print('Test F1 score: ', f1_score(y_test, rfc_pred))

In [ ]:
#plot confusion matrix of random forest using SMOTE oversampling
cnf_matrix_rf_smote_CV = confusion_matrix(y_test, rfc_pred)
print('Confusion Matrix:\n',cnf_matrix_rf_smote_CV)

In [ ]:
# save the model to disk
filename = 'rfc_model.sav'
pickle.dump(rf, open(filename, 'wb'))

<h3>XG Boost Classifier

In [70]:
from xgboost import XGBClassifier

In [73]:
xgb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', XGBClassifier()),
              ])
xgb.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))])

In [75]:
%%time

y_pred = xgb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
#print(classification_report(y_test, y_pred,target_names=targets))

accuracy 0.7563767964644883
CPU times: user 41.4 s, sys: 1.24 s, total: 42.6 s
Wall time: 43.7 s


In [77]:
import pickle

In [78]:
# save the model to disk
filename = 'xgboost_model.sav'
pickle.dump(xgb, open(filename, 'wb'))

In [79]:
xgb

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))])

<h3>BAG OF WORDS KERAS NEURAL NETWORK

In [41]:
import itertools
import os
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

Using TensorFlow backend.


In [42]:
#train test split
X = df.content
y = df.kmeans12

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 40)

In [43]:
#tokenize the unique words in the vocab
max_words = 1000

#limit the vocab to the top words (max_words)
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

#creates a word index lookup of the vocab
tokenize.fit_on_texts(X_train) # only fit on train

In [44]:
#create matrix to pass into the neural network
x_train = tokenize.texts_to_matrix(X_train)
x_test = tokenize.texts_to_matrix(X_test)

In [45]:
#encode labels and fit to training
encoder = LabelEncoder()
encoder.fit(y_train)

#transform labels
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [46]:
num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [47]:
batch_size = 32
epochs = 10

In [48]:
# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [49]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [50]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Instructions for updating:
Use tf.cast instead.
Train on 60347 samples, validate on 6706 samples
Epoch 1/10
60347/60347 [==============================] - 18s 299us/step - loss: 0.9146 - acc: 0.6738 - val_loss: 0.7230 - val_acc: 0.7377
Epoch 2/10
60347/60347 [==============================] - 18s 293us/step - loss: 0.7503 - acc: 0.7232 - val_loss: 0.7313 - val_acc: 0.7220
Epoch 3/10
60347/60347 [==============================] - 18s 293us/step - loss: 0.6991 - acc: 0.7383 - val_loss: 0.7108 - val_acc: 0.7338
Epoch 4/10
60347/60347 [==============================] - 18s 291us/step - loss: 0.6554 - acc: 0.7562 - val_loss: 0.7192 - val_acc: 0.7373
Epoch 5/10
60347/60347 [==============================] - 17s 289us/step - loss: 0.6116 - acc: 0.7686 - val_loss: 0.7226 - val_acc: 0.7302
Epoch 6/10
60347/60347 [==============================] - 18s 292us/step - loss: 0.5723 - acc: 0.7838 - val_loss: 0.7315 - val_acc: 0.7328
Epoch 7/10
60347/60347 [==============================] - 18s 293us/s

In [51]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])

28737/28737 [==============================] - 2s 60us/step
Test accuracy: 0.725510665692313


In [52]:
score

[0.8202775695957222, 0.725510665692313]

In [80]:
# save the model to disk
filename = 'keras_model.sav'
pickle.dump(history, open(filename, 'wb'))